In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 8.6MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 8.9MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


Wczytywanie danych


In [4]:
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

Feature Engineering

In [0]:
SUFFIX_CAT = "__cat"
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [6]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if "price" not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df["price_value"].values

  
  scores = cross_val_score(model, X, y, cv = 3, scoring = "neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

Decision Tree

In [8]:
run_model(DecisionTreeRegressor(max_depth=7), cat_feats)

(-17721.49757422326, 602.2105939403559)

Random Forest

In [11]:
model = RandomForestRegressor(max_depth=50, n_estimators=50, random_state=0)

run_model(model, cat_feats)


(-11172.849950405865, 286.97972427975753)

XGBoost

In [14]:
xgb_params = {
   "max_depth": 5,
   "n_estimators": 50,
   "learning_rate": 0.1,
   "seed": 0
}


run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[19:14:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:14:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:15:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [16]:
X = df[cat_feats].values
y = df["price_value"].values
m = xgb.XGBRegressor(max_depth = 5, n_estimators = 50, learning_rate = 0.1, seed = 0)
m.fit(X, y)

imp = PermutationImportance(m, random_state= 0).fit(X, y)
eli5.show_weights(imp, feature_names = cat_feats)

[19:16:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [18]:
feats = ["param_napęd__cat",
"param_rok-produkcji__cat",
"param_stan__cat",
"param_skrzynia-biegów__cat",
"param_faktura-vat__cat",
"param_moc__cat",
"param_marka-pojazdu__cat",
"feature_kamera-cofania__cat",
"param_typ__cat",
"param_pojemność-skokowa__cat",
"seller_name__cat",
"feature_wspomaganie-kierownicy__cat",
"param_model-pojazdu__cat",
"param_wersja__cat",
"param_kod-silnika__cat",
"feature_system-start-stop__cat",
"feature_asystent-pasa-ruchu__cat",
"feature_czujniki-parkowania-przednie__cat",
"feature_łopatki-zmiany-biegów__cat",
"feature_regulowane-zawieszenie__cat"]

len(feats)

20

In [21]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:23:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:23:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:23:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [22]:
df.param_napęd.unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [0]:
df["param_rok-produkcji"] = df["param_rok-produkcji"].map(lambda x: -1 if str(x) == "None" else int(x))

In [0]:
feats  = ["param_napęd__cat",
"param_rok-produkcji",
"param_stan__cat",
"param_skrzynia-biegów__cat",
"param_faktura-vat__cat",
"param_moc__cat",
"param_marka-pojazdu__cat",
"feature_kamera-cofania__cat",
"param_typ__cat",
"param_pojemność-skokowa__cat",
"seller_name__cat",
"feature_wspomaganie-kierownicy__cat",
"param_model-pojazdu__cat",
"param_wersja__cat",
"param_kod-silnika__cat",
"feature_system-start-stop__cat",
"feature_asystent-pasa-ruchu__cat",
"feature_czujniki-parkowania-przednie__cat",
"feature_łopatki-zmiany-biegów__cat",
"feature_regulowane-zawieszenie__cat"]

In [25]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:23:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:23:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:24:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [26]:
df["param_moc"].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [27]:
df["param_moc_1"] = df["param_moc"].map(lambda x: "-1" if str(x) == "None" else x.replace(" ", ""))

df["param_moc_2"] = df["param_moc_1"].map(lambda x: int(x.replace("KM", "")))
df["param_moc_2"]

0          90
2         115
3         262
4         110
5         310
         ... 
160609     75
160610     75
160611    120
160614    150
160615     88
Name: param_moc_2, Length: 106494, dtype: int64

In [0]:
feats  = ["param_napęd__cat",
"param_rok-produkcji",
"param_stan__cat",
"param_skrzynia-biegów__cat",
"param_faktura-vat__cat",
"param_moc_2",
"param_marka-pojazdu__cat",
"feature_kamera-cofania__cat",
"param_typ__cat",
"param_pojemność-skokowa__cat",
"seller_name__cat",
"feature_wspomaganie-kierownicy__cat",
"param_model-pojazdu__cat",
"param_wersja__cat",
"param_kod-silnika__cat",
"feature_system-start-stop__cat",
"feature_asystent-pasa-ruchu__cat",
"feature_czujniki-parkowania-przednie__cat",
"feature_łopatki-zmiany-biegów__cat",
"feature_regulowane-zawieszenie__cat"]




In [29]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:24:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:24:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:24:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9754.814038220027, 78.76301552059402)

In [30]:
df["param_pojemność-skokowa"]

0           898 cm3
2         1 560 cm3
3         3 000 cm3
4         1 560 cm3
5         1 984 cm3
            ...    
160609    1 120 cm3
160610    1 200 cm3
160611    1 900 cm3
160614    2 500 cm3
160615    1 360 cm3
Name: param_pojemność-skokowa, Length: 106494, dtype: object

In [31]:
df["param_pojemność-skokowa_1"] = df["param_pojemność-skokowa"].map(lambda x: "-1" if str(x) == "None" else x.replace(" ", ""))

df["param_pojemność-skokowa_2"] = df["param_pojemność-skokowa_1"].map(lambda x: int(x.replace("cm3", "")))
df["param_pojemność-skokowa_2"]

0          898
2         1560
3         3000
4         1560
5         1984
          ... 
160609    1120
160610    1200
160611    1900
160614    2500
160615    1360
Name: param_pojemność-skokowa_2, Length: 106494, dtype: int64

In [0]:
feats  = ["param_napęd__cat",
"param_rok-produkcji",
"param_stan__cat",
"param_skrzynia-biegów__cat",
"param_faktura-vat__cat",
"param_moc_2",
"param_marka-pojazdu__cat",
"feature_kamera-cofania__cat",
"param_typ__cat",
"param_pojemność-skokowa_2",
"seller_name__cat",
"feature_wspomaganie-kierownicy__cat",
"param_model-pojazdu__cat",
"param_wersja__cat",
"param_kod-silnika__cat",
"feature_system-start-stop__cat",
"feature_asystent-pasa-ruchu__cat",
"feature_czujniki-parkowania-przednie__cat",
"feature_łopatki-zmiany-biegów__cat",
"feature_regulowane-zawieszenie__cat"]


In [33]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:25:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:25:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:25:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9621.119663721702, 100.59307679134791)

In [39]:
X = df[feats].values
y = df["price_value"].values
model.fit(X, y)

imp = PermutationImportance(model, random_state= 0).fit(X, y)
eli5.show_weights(imp, feature_names = feats)

Weight,Feature
0.8990 ± 0.0188,param_rok-produkcji
0.4000 ± 0.0079,param_moc_2
0.1464 ± 0.0045,param_pojemność-skokowa_2
0.0405 ± 0.0016,param_marka-pojazdu__cat
0.0392 ± 0.0013,param_wersja__cat
0.0384 ± 0.0017,seller_name__cat
0.0324 ± 0.0004,param_model-pojazdu__cat
0.0315 ± 0.0015,param_napęd__cat
0.0246 ± 0.0007,param_typ__cat
0.0215 ± 0.0007,param_kod-silnika__cat


In [35]:
model = RandomForestRegressor(max_depth=50, n_estimators=100, random_state=0)

run_model(model, feats)

(-7910.228276344676, 91.09870873562613)